# Imports/Setup

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from IPython.display import clear_output
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

%matplotlib inline
load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="a6babb9c2f9c4878a49171de0e1c193e", client_secret="bbfb1d4aff7b4b5aba86acfd382125f3"))

# Download Data

In [2]:
genres = sp.recommendation_genre_seeds()['genres']
print(len(genres), 'genres')

126 genres


## Get Tracks

In [3]:
tracks = []
for i, g in enumerate(genres):
    clear_output(wait=True)
    print(i, g)
    recs = sp.recommendations(seed_genres=[g], limit=100)['tracks']
    recs_id_name_artist = [(track['id'], track['name'], track['artists'][0]['name'], g) for track in recs]
    for track in recs_id_name_artist:
        tracks.append(track)
clear_output(wait=True)
print('Tracks:', len(tracks))

Tracks: 12526


## Get Audio Features

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

track_chunks = list(chunks(tracks, 100))
audio_features = []
for c in track_chunks:
    c = [ch for ch in c if ch[0] is not None]
    c_ids = [track[0] for track in c]
    c_names = [track[1] for track in c]
    c_artists = [track[2] for track in c]
    c_genres = [track[3] for track in c]
    c_audio_features = sp.audio_features(tracks=c_ids)
    for i, _ in enumerate(c_audio_features):
        c_audio_features[i]['name'] = c_names[i]
        c_audio_features[i]['artist'] = c_artists[i]
        c_audio_features[i]['genre'] = c_genres[i]
    audio_features += c_audio_features
print('Audio Features:', len(audio_features))

Audio Features: 12526


## Save to CSV

In [5]:
track_df = pd.DataFrame(audio_features)
track_df.to_csv('../data/other.csv')
track_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name,artist,genre
0,0.692,0.564,3,-6.053,0,0.0918,0.4740,0.000000,0.1120,0.6370,...,audio_features,3GEKqgBUD2WQIsuVBv9rU0,spotify:track:3GEKqgBUD2WQIsuVBv9rU0,https://api.spotify.com/v1/tracks/3GEKqgBUD2WQ...,https://api.spotify.com/v1/audio-analysis/3GEK...,217133,4,American Boy - Radio 1 Live Lounge,Sam Sparro,acoustic
1,0.462,0.244,3,-16.960,1,0.0290,0.9510,0.173000,0.1200,0.0780,...,audio_features,02iZJRlOZ7VRbzO8uVdOva,spotify:track:02iZJRlOZ7VRbzO8uVdOva,https://api.spotify.com/v1/tracks/02iZJRlOZ7VR...,https://api.spotify.com/v1/audio-analysis/02iZ...,310733,4,In the Stream,S. Carey,acoustic
2,0.470,0.229,0,-12.371,1,0.0354,0.7010,0.000227,0.0838,0.2570,...,audio_features,26qhmNexrOJWTrakmx8ohR,spotify:track:26qhmNexrOJWTrakmx8ohR,https://api.spotify.com/v1/tracks/26qhmNexrOJW...,https://api.spotify.com/v1/audio-analysis/26qh...,207640,4,Creep,Karen Souza,acoustic
3,0.545,0.255,8,-14.844,1,0.0311,0.3620,0.000018,0.0841,0.8310,...,audio_features,2ZaYFNn1YQuLSVdHhanr4Q,spotify:track:2ZaYFNn1YQuLSVdHhanr4Q,https://api.spotify.com/v1/tracks/2ZaYFNn1YQuL...,https://api.spotify.com/v1/audio-analysis/2ZaY...,149013,4,Only You,Joshua Radin,acoustic
4,0.470,0.296,7,-14.123,1,0.0328,0.8530,0.368000,0.0957,0.0605,...,audio_features,2Uj2BjZXVv9WvYFEVl8TDi,spotify:track:2Uj2BjZXVv9WvYFEVl8TDi,https://api.spotify.com/v1/tracks/2Uj2BjZXVv9W...,https://api.spotify.com/v1/audio-analysis/2Uj2...,203827,4,Silent Way,Milo Greene,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12521,0.763,0.311,2,-13.650,1,0.0562,0.8680,0.895000,0.1040,0.1020,...,audio_features,5CV51opp4lfV2y0A0fDjn3,spotify:track:5CV51opp4lfV2y0A0fDjn3,https://api.spotify.com/v1/tracks/5CV51opp4lfV...,https://api.spotify.com/v1/audio-analysis/5CV5...,89227,4,Native American Drum Circle,Drums Of The World,world-music
12522,0.591,0.422,7,-13.389,0,0.0409,0.6180,0.952000,0.2180,0.6730,...,audio_features,7FIKMHuZDDlAx0j8wccvFj,spotify:track:7FIKMHuZDDlAx0j8wccvFj,https://api.spotify.com/v1/tracks/7FIKMHuZDDlA...,https://api.spotify.com/v1/audio-analysis/7FIK...,254027,4,Gomni,Ali Farka Touré,world-music
12523,0.785,0.743,9,-3.364,1,0.0441,0.0585,0.079900,0.1390,0.6980,...,audio_features,6nrUDtBBWBKuSDofJpphDd,spotify:track:6nrUDtBBWBKuSDofJpphDd,https://api.spotify.com/v1/tracks/6nrUDtBBWBKu...,https://api.spotify.com/v1/audio-analysis/6nrU...,263813,4,Epoca,Gotan Project,world-music
12524,0.511,0.662,11,-8.154,0,0.0378,0.8760,0.879000,0.0752,0.6510,...,audio_features,2KQhLhf8v7soTvYY57mXJ3,spotify:track:2KQhLhf8v7soTvYY57mXJ3,https://api.spotify.com/v1/tracks/2KQhLhf8v7so...,https://api.spotify.com/v1/audio-analysis/2KQh...,464000,4,Orion,Rodrigo y Gabriela,world-music


# DB Upload

In [6]:
conn = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(os.getenv('DB_USER'),
                                                               os.getenv('DB_PASS'), 
                                                               os.getenv('DB_IP'), 
                                                               os.getenv('DB_PORT'), 
                                                               os.getenv('DB_NAME')))
track_df.to_sql('other', con=conn, if_exists='replace')

ValueError: invalid literal for int() with base 10: 'None'